In [3]:
import numpy
import time
import ipyparallel as ipp
import qiskit.providers.aer.noise as noise
from scipy.stats import multinomial
from scipy import stats
from joblib import Parallel, delayed,Memory
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
#from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.chemistry import FermionicOperator
from qiskit.providers.aer import AerSimulator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit.quantum_info import Pauli
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation, CircuitSampler, CircuitStateFn 
import scipy
from joblib import Parallel, delayed
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.compiler import transpile
from qiskit.circuit.random import random_circuit
from qiskit.chemistry.components.variational_forms import UCCSD

In [4]:
import warnings
warnings. filterwarnings('ignore')

In [5]:
c =ipp.Client()
num_qubits=8

In [37]:
%run Utils.ipynb
%run optimizers.ipynb
%run EstimateHamiltonianMean.ipynb

In [27]:
U=7
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)

In [28]:
#Hamiltonian grouping for circuit based measurement
eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U)
qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U)
opArr=[qubitH_0,qubitH_1,qubitH_2]
#two body off diags
circ0_1gate,circ0_1gate_inv,rotH2body=OffDiag2BodyTerms(opArr[2],send_inverse=True)
#one body off diags
circ0_8gate,circ0_8gate_inv,rotH1body=OffDiag1BodyTerms(opArr[0],send_inverse=True)
Ops=[rotH1body,opArr[1],rotH2body]

In [29]:
U=7
backend=Aer.get_backend('statevector_simulator')
HFcirc,HFstate=HF_init_state(U)
var_form_base=UCCSD(8,num_particles=4, initial_state=HFcirc,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
var_form_base.manage_hopping_operators()

Final Hartree Fock Energy -18.788335371650774


In [30]:
qr=QuantumRegister(8,'q')

In [38]:
noise_model,basis_gates=NoiseModel()

NoiseModel:
  Basis gates: ['cx', 'id', 'u1', 'u2', 'u3']
  Instructions with noise: ['u1', 'measure', 'u2', 'cx', 'u3']
  All-qubits errors: ['measure', 'u1', 'u2', 'u3', 'cx']


In [32]:
#Control Hamiltonian at lower U value
eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U-3)
qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U-3)
opArr=[qubitH_0,qubitH_1,qubitH_2]
#two body off diags
circ0_1gate_ctrl,rotH2body=OffDiag2BodyTerms(opArr[2])
#one body off diags
circ0_8gate_ctrl,rotH1body=OffDiag1BodyTerms(opArr[0])
Ops_ctrl=[rotH1body,opArr[1],rotH2body]

In [40]:
HamiltonianEstm([])-Energy([])

mean -18.247781398206595 abs err 0.5405539734441795 (-18.788335371650774+0j)


array([0.5405539734441795, 18.79116933439186,
       array([18.27148982,  1.05714518, 18.78858972]),
       array([18.78925712+0.j, 18.78962804+0.j, 18.78895492+0.j]),
       array([20.34621315, 27.78888063, 23.80026465])], dtype=object)

In [58]:
Hestimate([],basis_gates=basis_gates,noise_model=noise_model)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    4.8s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:   10.9s
[Parallel(n_jobs=7)]: Done  90 out of  90 | elapsed:   26.5s finished


-18.493164198428538

In [42]:
Energy([])

-18.788335371650774

In [76]:
qubitH_aux=qubitH_0+qubitH_1+qubitH_2

In [88]:
circ=var_form_base.construct_circuit(parameters=[])
state=execute(circ,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()

In [93]:
M=qubitH_0.to_matrix(),qubitH_1.to_matrix(),qubitH_2.to_matrix()
state@M[0]@state.real,state@M[1]@state.real,state@M[2]@state.real

((-0.8019567983235492+0j),
 (-10.471532010724655+0j),
 (2.629610465479082e-15+0j))

In [90]:
-0.9403880104418357-1.7435722666024656-1.7499999999999996

-4.433960277044301